Homework 4:
---
For this homework we will be building various neural networks to classify handwritten digits.

### Question 1: Feed-Forward Neural Network to classify MNIST digits.

Train a neural network to classify MNIST digits. Remember we have to pick 3 different components to build a neural network: (1) Network Architecture (2) Loss Function and (3) Optimizer. For this assignment use the following:

Network Architecture:
* [28x28] input 
* -> Use a view to reshape to [784]
* -> Linear Layer [784, Num Hidden Units]
* -> Relu
* -> Output Linear Layer [NumHiddenUnits, 10]
* -> Log Softmax (`torch.nn.functional.log_softmax(output)`)

Loss: Please use the negative log likelihood loss function (nll_loss).
Optimizer: Please use SGD optimizer (https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD), but you can also experiment with Adam optimizer in 1.6.

1.1: Create a Pytorch module to implement a version of this neural network. Display the shape of the network using ` torchsummary.summary`.

1.2 Create a training loop to train this model using SGD optimizer. Note, we should train using minibatches. The dataloader will create minibatches for you automatically and you can just loop through as follows:

```
for epoch in range(0, num_epochs):
  # Loop over the dataloader and get minibatches:
  for batch_idx, (data, target) in enumerate(train_dataloader):
    # TODO: Reset gradient on optimizer
    # TODO: Evaluate th emodel on data
    # TODO: Calculate loss using nll_loss
    # TODO: Evaluate gradient with respect to the loss function
    # TODO: Run a step of the optimizer.
    # TODO: Store the loss function for plotting every X batches.
```


1.3 Train your model on MNIST digits. Please record the training loss every 100 minibatch steps. Display this as a plot where the x axis is the minibatch index and the y axis is the training loss at that step.

1.4: Create a classifier for your model. The output of your model can be evaluated by usuing the forward function, or just calling the model ```trained_nn_model(sample)```. The output of your model will be a tensor of 10 values representing the log softmax value for each class. You can build a classifier by simply taking the index with the highest value. For example if the classifier outputs `[.1, .2, .6, .1]` you would classify it as class 2.

1.5: Create and display a confusion matrix for your classifier on your Train and Test data using https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html. Discuss what you observe from this confusion matrix. Which digits are mistaken for each other?

1.6: Experiment with different network shapes (number of hidden layers and number of hidden units per layer) or optimizers (for example, you can try Adam instead of SGD). Report the Train and Test accuracy of at least 4 experiments here and discuss what works well and what does not.

1.7: Find some examples of images your model misclassifies. Display these images and discuss why the model does not work well on them. 

### Question 2: Convolutional Neural Network to classify MNIST digits.

We will answer similar questions as above but using a convolutional neural network. Please start with this example network module.

```
class SimpleConvNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.dense_layer = nn.Linear(320, 50)      
        self.output_layer = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.dense_layer(x))
        x = F.dropout(x, training=self.training)
        output = self.output_layer(x)
        return F.log_softmax(output)
```

2.1: Create a Pytorch module to implement a version of this neural network as implemented above. Display the shape of the network using ` torchsummary.summary`.

2.2 Create a training loop to train this model using Adam optimizer. This should be similar to 1.2.

2.3 Train your model on MNIST digits. Please record the training loss every 100 minibatch steps. Display this as a plot where the x axis is the minibatch index and the y axis is the training loss at that step.

2.4: Create a classifier for your model and report train and test accuracy. How does this compare to the performance of your non-convolutional neural network in question 1?

2.5: Create and display a confusion matrix for your classifier on your Train and Test data. Discuss what you observe from this confusion matrix. Which digits are mistaken for each other? How does this differ from 1.5?

2.6: Experiment with different kernel sizes and stride parameters to your convolutions. Report the Train and Test accuracy of at least 2 experiments here and discuss what works well and what does not.

2.7: Experiment with the network structure. Can you add in an additional convolution? Report the Train and Test accuracy of at least 2 experiments here and discuss what works well and what does not.

In [ ]:
# Imports
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.pyplot import figure
from matplotlib import cm
import numpy as np

In [ ]:
# Load the MNIST dataset:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_train_dataset = torchvision.datasets.MNIST(
    root='../data/mnist', train=True, download=True, transform=transform) 
mnist_test_dataset = torchvision.datasets.MNIST(
    root='../data/mnist', train=False, download=True, transform=transform)

In [ ]:
def image_grid(data, num_figs):
    """Function to display a grid of images
    Args:
      data: torchvision dataset
      num_figs: number of images to display
    """
    fig = figure(figsize=(10, 8), dpi=80)
    for i in range(num_figs):
        fig_i = int(math.sqrt(num_figs))
        plt.subplot(fig_i, fig_i, i+1)
        plt.tight_layout()
        image, label = data[i]
        plt.imshow(image.reshape(28, 28), cmap='gray')
        plt.title(f"Label {label}")
        plt.xticks([])
        plt.yticks([])
image_grid(mnist_train_dataset, 25)

In [ ]:
# Let's set the random seed so we will get the seame behavior each run.
random_seed = 42
torch.manual_seed(random_seed)

# Let's make sure to turn off GPU backend in case you happen to have GPU. The only
# reason we do this is not to have to worry about moving data and models on and
# off of the GPU for now. If you are training large models and have a GPU, then
# you will want to use it.
torch.backends.cudnn.enabled = False

In [ ]:
# Put training and test data into data loaders allow batching of datasets.
batch_size_train = 64
batch_size_test = 1000
train_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=batch_size_test, shuffle=True)

In [ ]:
class SimpleNeuralNet(nn.Module):
    def __init__(self, hidden_units):
        super().__init__()
        # TODO

    def forward(self, x):
        # TODO
nn_model = SimpleNeuralNet(num_hidden_units)
summary(nn_model, (1, 28, 28))